### Cleaning the Survey Dataset
As a data analyst, it is your responsibility to ensure data is clean, consistent and accurate. In this notebook, I focused on cleaning a survey dataset containing salary information for managers across various career fields, collected between 2021 and 2024. The data cleaning process was carried out using Python libraries such as Pandas, NumPy and FuzzyWuzzy.

In [1]:
#importing the library
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from fuzzywuzzy import process

In [2]:
#importing the dataset
df=pd.read_csv("Salary Survey.csv")

### Data Exploration

In [3]:
#lets get to know our data
display(df.head())
print(f"\nShape :{df.shape}")
print(f"\nData Types: {df.dtypes}")

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White



Shape :(5236, 18)

Data Types: Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                                                object
If your 

### Data Preprocessing
The first step involves renaming the columns to enhance visibility and improve readability.

In [4]:
#renaming the columns
df.columns=[
    "Timestamp",
    "Age",
    "Job Industry",
    "Job Title",
    "Job Specifics",
    "Annual Salary",
    "Monetary Compensation",
    "Currency",
    "Other Currency",
    "Income Specifics",
    "Occupation Country",
    "US States",
    "Occupation City",
    "Overall Experience",
    "Experience",
    "Education Level",
    "Gender",
    "Race"
]
#lets display our new dataset
display(df.head())

,Timestamp,Age,Job Industry,Job Title,Job Specifics,Annual Salary,Monetary Compensation,Currency,Other Currency,Income Specifics,Occupation Country,US States,Occupation City,Overall Experience,Experience,Education Level,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


* Lets check for missing values

In [5]:
#checking for missing values
df.isna().sum()

Timestamp                   0
Age                         0
Job Industry               10
Job Title                   0
Job Specifics            3695
Annual Salary               0
Monetary Compensation    1339
Currency                    0
Other Currency           5210
Income Specifics         4553
Occupation Country          0
US States                 792
Occupation City            17
Overall Experience          1
Experience                  1
Education Level            34
Gender                     25
Race                       20
dtype: int64

* After examining the missing values, I identified some columns that need to be dropped entirely, while certain rows in other columns will also need to be removed

In [6]:
#dropping the columns with missing values
df.drop(columns=["Job Specifics","Other Currency","Income Specifics","US States"],inplace=True)
#dropping the missing values
df.dropna(inplace=True)

* I will extract the Year column, representing when the survey was conducted, from the Timestamp column and then drop the Timestamp column as it will no longer be needed. Before doing so, I will first change the data type of the Timestamp column.

In [7]:
#lets change the data type
df["Timestamp"]=pd.to_datetime(df["Timestamp"])

#getting the Year column
df["Record Year"]=df["Timestamp"].dt.year

#dropping the Timestamp column
df.drop(columns="Timestamp",inplace=True)

* We'll create a new column, Total Annual Salary, by summing the values in the Annual Salary and Monetary Compensation columns. However, before this, we'll clean these columns by correcting errors and converting their data types appropriately.

In [8]:
#lets clean the Annual Salary column
df["Annual Salary"]=df["Annual Salary"].str.replace(r"[^\d]","",regex=True)
df["Annual Salary"]=df["Annual Salary"].astype(int)
#changing the data type of AnnuaL Salary
df["Annual Salary"]=df["Annual Salary"].astype(int)
#changing the data type of Monetary Compensation
df["Monetary Compensation"]=df["Monetary Compensation"].astype(int)
#getting the Annual Salary column
df["Total Annual Salary"]=df["Annual Salary"] + df["Monetary Compensation"]

* Next step is adding the currency signs to the Annual Salary, Total Annual Salary and Monetary Compensation. After that I will drop the Currency column.

In [9]:
df["Currency"].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'Other', 'CHF', 'ZAR',
       'HKD', 'SEK'], dtype=object)

In [10]:
df["Annual Salary"]=df["Currency"] + " " + df["Annual Salary"].astype(str)
df["Monetary Compensation"]=df["Currency"] + " " + df["Monetary Compensation"].astype(str)
df["Total Annual Salary"]=df["Currency"] + " " + df["Total Annual Salary"].astype(str)
#dropping the currency column
df.drop(columns="Currency",inplace=True)

* I will create categories for the Age column to enhance interpretability.

In [11]:
df["Age"].unique()

array(['25-34', '45-54', '35-44', '18-24', '65 or over', '55-64',
       'under 18'], dtype=object)

In [12]:
#lets create the function that will clean
def age_group(age):
    if age =="18-24":
        return "Young"
    elif age =="25-34":
        return "Mid Life"
    elif age =="35-44":
        return "Junior"
    elif age =="45-54":
        return "Experienced"
    elif age =="55-64":
        return "Senior"
    else:
        return "Retired"
df["Age Group"]=df["Age"].apply(age_group)

#lets drop the Age column since we no longer need it
df.drop(columns="Age",inplace=True)

* I will create categories for the Overall Experience column to enhance interpretability.

In [13]:
df["Overall Experience"].unique()

array(['5-7 years', '8 - 10 years', '2 - 4 years', '21 - 30 years',
       '11 - 20 years', '41 years or more', '31 - 40 years',
       '1 year or less'], dtype=object)

In [14]:
#lets create a function to clean the Experience column
def experience(year):
    if year =="1 year or less":
        return "Entry Level"
    elif year =="2 - 4 years":
        return "Junior Level"
    elif year =="5 - 7 years":
        return "Mid Level"
    elif year == "8 - 10 years":
        return "Experienced Level"
    elif year == "11 - 20 years":
        return "Senior Level"
    elif year == "21 - 30 years":
        return "Veteran Level"
    elif year == "31 - 40 years":
        return "Expert Level"
    else:
        return "Legendary Level"
df["Experience Level"]=df["Overall Experience"].apply(experience)

#lets drop the Experience and Overall Experience columns
df.drop(columns=["Experience","Overall Experience"],inplace=True)

* The Race column seems to have some inconsistencies, so I will have to clean it.

In [15]:
df["Race"].unique()

array(['White', 'Hispanic, Latino, or Spanish origin, White',
       'Asian or Asian American, White', 'Asian or Asian American',
       'Another option not listed here or prefer not to answer',
       'Middle Eastern or Northern African',
       'Hispanic, Latino, or Spanish origin', 'Black or African American',
       'Black or African American, Hispanic, Latino, or Spanish origin, White',
       'Native American or Alaska Native, White',
       'Hispanic, Latino, or Spanish origin, Another option not listed here or prefer not to answer',
       'White, Another option not listed here or prefer not to answer',
       'Black or African American, Native American or Alaska Native, White',
       'Asian or Asian American, Another option not listed here or prefer not to answer',
       'Middle Eastern or Northern African, White',
       'Black or African American, White',
       'Asian or Asian American, Black or African American, White',
       'Black or African American, Hispanic, Latino

In [16]:
#lets create the function to perform the cleaning
def clean_race(value):
    if "White" in value and "," not in value:
        return "White"
    elif "Black or African American" in value and "," not in value:
        return "Black or African American"
    elif "Asian or Asian American" in value and "," not in value:
        return "Asian or Asian American"
    elif "Hispanic, Latino, or Spanish origin" in value and "," not in value:
        return "Hispanic, Latino, or Spanish origin"
    elif "Native American or Alaska Native" in value and "," not in value:
        return "Native American or Alaska Native"
    elif "Middle Eastern or Northern African" in value and "," not in value:
        return "Middle Eastern or Northern African"
    elif "Another option not listed here or prefer not to answer" in value:
        return "Other or Prefer not to answer"
    else:
        return "Multiracial"

df["Race"] = df["Race"].apply(clean_race)

* Also the Gender column seems to have some inconsistencies, so I will have to clean it.

In [17]:
df["Gender"].unique()

array(['Woman', 'Non-binary', 'Man', 'Other or prefer not to answer'],
      dtype=object)

In [18]:
#lets create a function that will perform the cleaning
def gender(value):
    if value =="Woman":
        return "Female"
    elif value =="Man":
        return "Male"
    elif value =="Non-binary":
        return "Non Binary"
    elif value =="Other or prefer not to answer":
        return "Other"
    else:
        return "Prefer not to answer"
        
df["Gender"]=df["Gender"].apply(gender)

* The Occupation Country has alot of error that need cleaninng clean.

In [19]:
df["Occupation Country"].unique()

array(['United States', 'United Kingdom', 'USA', 'US', 'United Kingdom ',
       'UK', 'Canada', 'United States ', 'The Netherlands', 'Australia ',
       'us', 'Usa', 'United States of America', 'France', 'United states',
       'USA ', 'Germany', 'U.S.', 'UK ', 'united states', 'Ireland',
       'Scotland', 'usa', 'India', 'Australia', 'Uk',
       'United States of America ', 'U.S. ', 'Canada ', 'U.S>', 'ISA',
       'Argentina', 'US ', 'United State', 'U.S.A', 'England', 'Denmark',
       'America', 'Netherlands', 'netherlands', 'Spain', 'England ',
       'U.S.A.', 'Switzerland', 'Bermuda', 'Us', 'The United States',
       'Malaysia', 'United Stated', 'South Africa ', 'Belgium',
       'Northern Ireland', 'u.s.', 'UNITED STATES', 'United states ',
       'Hong Kong', 'Kuwait', 'Norway', 'Contracts',
       'USA-- Virgin Islands', 'United Statws', 'U.S',
       "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise

* First I will create a dictionary that maps the incorrect or inconsistent values to the correct ones. This took some time because I had to do it manually.
#### Note
I would be grateful if someone had a good idea on how I would have cleaned it better.

In [20]:
#lets first remove the leading and trailing white spaces
df["Occupation Country"]=df["Occupation Country"].str.strip()

In [21]:
country_mapping = {
    # United States
    "United States": "United States", "USA": "United States", "US": "United States", "U.S.": "United States",
    "America": "United States", "United states": "United States", "United States of America": "United States",
    "U.S.A": "United States", "United State": "United States", "U.S.A.": "United States",
    "UNITED STATES": "United States", "Unted States": "United States", "United Sates": "United States",
    "United Statws": "United States", "Unites States": "United States", "Uniited States": "United States", "USA-- Virgin Islands": "United States",
    "Usa": "United States",

    # United Kingdom
    "United Kingdom": "United Kingdom", "UK": "United Kingdom", "England": "United Kingdom",
    "Great Britain": "United Kingdom", "Britain": "United Kingdom", "Scotland": "United Kingdom",
    "Northern Ireland": "United Kingdom", "England, UK.": "United Kingdom", "United kingdom": "United Kingdom", "Uk": "United Kingdom",

    # Canada
    "Canada": "Canada", "CANADA": "Canada", "Canada, Ottawa, ontario": "Canada", "Canadw": "Canada",
    "canada": "Canada",

    # Australia
    "Australia": "Australia", "Australia ": "Australia",

    # Netherlands
    "The Netherlands": "Netherlands", "Netherlands": "Netherlands", "netherlands": "Netherlands",
    
    # Other countries
    "Germany": "Germany", "germany": "Germany", "Germany ": "Germany",
    "France": "France", "FRANCE": "France",
    "India": "India",
    "Spain": "Spain",
    "South Africa": "South Africa", "South Africa ": "South Africa",
    "ireland": "Ireland"
}


In [22]:
#lets create a list of the unique countries
correct_countries = list(set(country_mapping.values()))
#lets define our function that will help us match the correct countries
def fuzzy_match(country, choices, threshold=80):
    match, score = process.extractOne(country, choices)
    return match if score >= threshold else country

df["Occupation Country"] = df["Occupation Country"].apply(
    lambda x: fuzzy_match(x, correct_countries) if x not in correct_countries else x
)

In [23]:
df["Occupation Country"].unique()

array(['United States', 'United Kingdom', 'USA', 'Australia', 'UK',
       'Canada', 'Netherlands', 'Usa', 'France', 'Germany', 'Ireland',
       'Scotland', 'usa', 'India', 'Uk', 'ISA', 'Argentina', 'U.S.A',
       'England', 'Denmark', 'America', 'Spain', 'U.S.A.', 'Switzerland',
       'Bermuda', 'Malaysia', 'South Africa', 'Belgium', 'Hong Kong',
       'Kuwait', 'Norway', 'Contracts', 'USA-- Virgin Islands',
       "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located",
       'England, UK.', 'Britain', 'Brazil', 'Sweden',
       'Worldwide (based in US but short term trips aroudn the world)',
       'Great Britain', 'Mexico', 'Trinidad and Tobago', 'Cayman Islands',
       'I am located in Canada but I work for a company in the US',
       'U.A.'], dtype=object)

* The Occupation Country column still requires for cleaning. I will create a function to clean.

In [24]:
#defining the function
def cleaned_country(value):
    if value in ["Usa", "usa", "U.S.A", "USA-- Virgin Islands", "USA", "U.S.A.", "America"]:
        return "United States"
    elif value in ["UK", "Uk", "Britain", "Great Britain", "England", "England, UK."]:
        return "United Kingdom"
    elif value in ["Contracts", "U.A.", "ISA"]:
        return np.nan
    else:
        return value

df["Occupation Country"] = df["Occupation Country"].apply(cleaned_country)

In [25]:
df["Occupation Country"].unique()

array(['United States', 'United Kingdom', 'Australia', 'Canada',
       'Netherlands', 'France', 'Germany', 'Ireland', 'Scotland', 'India',
       nan, 'Argentina', 'Denmark', 'Spain', 'Switzerland', 'Bermuda',
       'Malaysia', 'South Africa', 'Belgium', 'Hong Kong', 'Kuwait',
       'Norway',
       "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located",
       'Brazil', 'Sweden',
       'Worldwide (based in US but short term trips aroudn the world)',
       'Mexico', 'Trinidad and Tobago', 'Cayman Islands',
       'I am located in Canada but I work for a company in the US'],
      dtype=object)

* Lets further clean the column to remove the inconsistencies

In [26]:
invalid_entries = [
     "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located",
     'Worldwide (based in US but short term trips aroudn the world)',
     'I am located in Canada but I work for a company in the US'
]
df["Occupation Country"] = df["Occupation Country"].replace(invalid_entries, np.nan)

* Recall that some of the countries I had changed them to Nan, thus I will need to drop them.

In [27]:
df.dropna(subset="Occupation Country",inplace=True)

* Lets check if our dataset is now tidy

In [28]:
print(f"\nDataset Shape: {df.shape}")
print(f"\nMissing data: {df.isna().any()}")
print(f"\nData types: {df.dtypes}")


Dataset Shape: (3839, 13)

Missing data: Job Industry             False
Job Title                False
Annual Salary            False
Monetary Compensation    False
Occupation Country       False
Occupation City          False
Education Level          False
Gender                   False
Race                     False
Record Year              False
Total Annual Salary      False
Age Group                False
Experience Level         False
dtype: bool

Data types: Job Industry             object
Job Title                object
Annual Salary            object
Monetary Compensation    object
Occupation Country       object
Occupation City          object
Education Level          object
Gender                   object
Race                     object
Record Year               int32
Total Annual Salary      object
Age Group                object
Experience Level         object
dtype: object


* Lets check our new dataset

In [29]:
df.head()

,Job Industry,Job Title,Annual Salary,Monetary Compensation,Occupation Country,Occupation City,Education Level,Gender,Race,Record Year,Total Annual Salary,Age Group,Experience Level
0,Education (Higher Education),Research and Instruction Librarian,USD 55000,USD 0,United States,Boston,Master's degree,Female,White,2021,USD 55000,Mid Life,Legendary Level
1,Computing or Tech,Change & Internal Communications Manager,GBP 54600,GBP 4000,United Kingdom,Cambridge,College degree,Non Binary,White,2021,GBP 58600,Mid Life,Experienced Level
3,Nonprofits,Program Manager,USD 62000,USD 3000,United States,Milwaukee,College degree,Female,White,2021,USD 65000,Mid Life,Experienced Level
4,"Accounting, Banking & Finance",Accounting Manager,USD 60000,USD 7000,Australia,Greenville,College degree,Female,White,2021,USD 67000,Mid Life,Experienced Level
6,Publishing,Publishing Assistant,USD 33000,USD 2000,United States,Columbia,College degree,Female,White,2021,USD 35000,Mid Life,Junior Level


* I will rearrange the columns to ensure a more logical and organized order by adjusting their positions.

In [30]:
cols_move=[("Record Year",0),("Job Industry",6),("Job Title",7),("Experience Level",9),("Education Level",8),("Gender",3),("Annual Salary",11),("Monetary Compensation",10),("Total Annual Salary",12),("Age Group",1),("Occupation Country",5),("Occupation City",4),("Race",2)]
cols=list(df.columns)
for col,position in cols_move:
    cols.remove(col)
    cols.insert(position,col)
df=df[cols]    

In [31]:
df.head()

,Record Year,Age Group,Race,Gender,Job Industry,Occupation City,Occupation Country,Job Title,Education Level,Experience Level,Monetary Compensation,Annual Salary,Total Annual Salary
0,2021,Mid Life,White,Female,Education (Higher Education),Boston,United States,Research and Instruction Librarian,Master's degree,Legendary Level,USD 0,USD 55000,USD 55000
1,2021,Mid Life,White,Non Binary,Computing or Tech,Cambridge,United Kingdom,Change & Internal Communications Manager,College degree,Experienced Level,GBP 4000,GBP 54600,GBP 58600
3,2021,Mid Life,White,Female,Nonprofits,Milwaukee,United States,Program Manager,College degree,Experienced Level,USD 3000,USD 62000,USD 65000
4,2021,Mid Life,White,Female,"Accounting, Banking & Finance",Greenville,Australia,Accounting Manager,College degree,Experienced Level,USD 7000,USD 60000,USD 67000
6,2021,Mid Life,White,Female,Publishing,Columbia,United States,Publishing Assistant,College degree,Junior Level,USD 2000,USD 33000,USD 35000


* Now the dataset is tidy,lets save it

In [32]:
df.to_csv(r"C:\Users\danalyst\Desktop\Projects\Data-Cleaning\Tidy_Salary_Survey.csv")